# 0.0. IMPORTS

In [1]:
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile
import csv
import pandas as pd
import warnings
import math
import translators as ts
import emoji
import demoji

#TWITTER
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import os

C:\Users\samuel.soares\Anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)
Using state Goias server backend.


## 0.1. Função Para Visualização Jupyter

In [3]:
import seaborn           as sns
from IPython.core.display  import HTML
def jupyter_settings():
    %matplotlib inline
    %pylab inline
 
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [20,10]
    plt.rcParams['font.size'] = 24
  
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 100 
    pd.set_option( 'display.expand_frame_repr', False )
  
    sns.set()

jupyter_settings()

warnings.filterwarnings('ignore') #desativar os warnings nos outputs
pd.set_option('display.max_row', 5000) #aparecer 3000 linhas de um dataframe para melhor visualização do conjunto todo
%config IPCompleter.greedy=True #extensão para autocomplementar o código

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\samuel.soares\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# 1. COLETA DE DADOS DOS CLIENTES

In [4]:
df_dados_clientes = pd.read_excel ('Dados de Clientes.xlsx')

## 1.1. Usuários do Instagram

In [5]:
municipios = df_dados_clientes['Município']
usuarios = df_dados_clientes['Usuario Instagram']
estados = df_dados_clientes['UF']

# 2. EXTRAINDO DADOS DO INSTAGRAM

## 2.1. Funções de extração do Instagram


In [6]:
def seguidores_usuario(L,user_name):
    profile = instaloader.Profile.from_username(L.context, user_name)
    seguidores = profile.followers
        
    return seguidores

def seguidos_pelo_usuario(L,user_name):
    profile = instaloader.Profile.from_username(L.context, user_name)
    seguindo = profile.followees
        
    return seguindo         

## 2.2. Funções Análise de Sentimento


In [78]:
#Função para limpar o texto

def cleanTxt(texto):
    texto = re.sub(r'@[a-zA-Z0-9]+', '', texto) #Remove menções
    texto = re.sub(r'#', '', texto) #Remove as hashtags
    texto = re.sub(r'RT[\s]+', '', texto) #Remove RTs
    texto = re.sub(r'https?:\/\/\S+', '', texto) #Remove hiperlinks
    texto = re.sub(r'\n', ' ', texto) #Remove caracter de nova linha
    texto = re.sub(' {2,}', '', texto).strip(' ')
    texto = re.sub(' {3,}', '', texto).strip(' ')
    
    return texto

#Função para extrair os emojis
def extract_emojis(s):
    return ''.join((' '+c+' ') if c in emoji.EMOJI_DATA else c for c in s)

#Função para remover emojis
def remove_emojis(data):
    emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

#Função para remover emojis duplicados
def remove_duplicate_emoji(orig_str):
    prev_emoji = None
    remove_duplicate_emoji = []
    for c in orig_str:
        if c in emoji.EMOJI_DATA:
            if prev_emoji == c:
                continue
            prev_emoji = c
        remove_duplicate_emoji.append(c)
    return "".join(remove_duplicate_emoji)

#Função para subjetividade
def Subjetividade(texto):
    return TextBlob(texto).sentiment.subjectivity

#Função para polaridade
def Polaridade(texto):
    tb = TextBlob(texto)
    #tb_en = tb.translate(to='en', from_lang='pt')
    return tb.sentiment.polarity

#Função para polaridade com tradução
def Polaridade_traducao(texto):
    tb = TextBlob(texto)
    tb_en = tb.translate(to='en', from_lang='pt')
    return tb_en.sentiment.polarity

#Função para análises positivas, neutras e negativas
def getAnalysis(score):
    if score < 0:
        return 'Negativo'
    elif score == 0:
        return 'Neutro'
    else:
        return 'Positivo'

## 2.5. Análise de sentimento do Instagram

In [15]:
#Abrir arquivo com rankings de emojis
df_Ranking_Emojis = pd.read_csv('Data/ranking_emojis.csv', delimiter = ',')
df_Ranking_Emojis['unicode'] = df_Ranking_Emojis['unicode'].apply(lambda x: re.sub("b'\\\\\\\\", '', x))
df_Ranking_Emojis['unicode'] = df_Ranking_Emojis['unicode'].apply(lambda x: re.sub("'", '', x))

In [68]:
df = pd.read_csv('Data/TABELA_POSTS_COMENTARIOS_MUNICIPIOS.csv', delimiter = ',')
df_coment = df.copy()
df_coment['Emojis'] = None
df_coment['Texto'] = None
df_coment['Unicodes'] = None
df_coment = df_coment[df_coment['Comentarios'].notnull()]
df_coment['Comentarios'] = df_coment['Comentarios'].apply(extract_emojis)
df_coment['Comentarios'] = df_coment['Comentarios'].apply(lambda x: x.split())

for i in range(len(df_coment['Comentarios'])):
    j=0
    for j in range(len(df_coment['Comentarios'].values[i])):
        if (df_coment['Comentarios'].values[i][j].startswith('@')):
            df_coment['Comentarios'].values[i][j] = ''

for k in range( len(df_coment['Comentarios']) ): 
    l=0
    lista_emojis = []
    lista_texto = []
    for l in range( len(df_coment['Comentarios'].values[k]) ):
        if ( df_coment['Comentarios'].values[k][l] in emoji.EMOJI_DATA ):
            lista_emojis.append(str(df_coment['Comentarios'].values[k][l]))
            
        else:
            lista_texto.append(str(df_coment['Comentarios'].values[k][l]))

    df_coment['Texto'].values[k] = lista_texto
    df_coment['Emojis'].values[k] = lista_emojis

for g in range( len(df_coment['Emojis']) ):
    h=0
    lista_unicodes = []
    for h in range( len(df_coment['Emojis'].values[g]) ):
        aux_1 = df_coment['Emojis'].values[g][h]
        aux_2 = aux_1.encode('unicode_escape')
        lista_unicodes.append(str(aux_2))

    df_coment['Unicodes'].values[g] = lista_unicodes


df_coment['Unicodes'] = df_coment['Unicodes'].apply(lambda x: ' '.join(x))
df_coment['Unicodes'] = df_coment['Unicodes'].apply(lambda x: re.sub("b'\\\\\\\\", '', x))
df_coment['Unicodes'] = df_coment['Unicodes'].apply(lambda x: re.sub("'", '', x))
df_coment['Unicodes'] = df_coment['Unicodes'].apply(lambda x: x.split())
df_coment['Texto'] = df_coment['Texto'].apply(lambda x: ' '.join(x))
df_coment['Comentarios'] = df_coment['Comentarios'].apply(lambda x: ' '.join(x))
df_coment['Emojis'] = df_coment['Emojis'].apply(lambda x: ''.join(x))
df_coment['Texto'] = df_coment['Texto'].apply(cleanTxt)
df_coment['ComentarioTraduzido'] = df_coment['Texto'].apply(lambda x: ts.google(x, from_language='pt', to_language='en') )#if (x != '') else x)

for a in range( len(df_coment['Emojis']) ):
    b=0
    list_nao_repetidos =[]
    for b in range( len(df_coment['Emojis'].values[a]) ):
        if (df_coment['Emojis'].values[a][b] not in list_nao_repetidos):
            list_nao_repetidos.append(df_coment['Emojis'].values[a][b])
            
    df_coment['Emojis'].values[a] = list_nao_repetidos

for c in range(0, len(df_coment['Unicodes']) ):
    d=0
    contador = 0
    for d in range(0, len(df_coment['Unicodes'].values[c]) ):
        e = 0
        for e in range ( len(df_Ranking_Emojis['unicode']) ):
            if ( df_coment['Unicodes'].values[c][d] == df_Ranking_Emojis['unicode'].values[e] ):
                score = df_Ranking_Emojis['score'].values[e]

                if (score < 0.0):
                    df_coment['Polaridade'].values[c] -= 0.2

                elif (score >= 0 and score < 0.2):
                    df_coment['Polaridade'].values[c] -= 0.1

                elif (score >= 0.2):
                    df_coment['Polaridade'].values[c] += 0.2

                else:
                    pass

            else :
                pass

df_coment['Subjetividade'] = df_coment['ComentarioTraduzido'].apply(Subjetividade)
df_coment['Polaridade'] = df_coment['ComentarioTraduzido'].apply(Polaridade)

df_coment['Analise'] = df_coment['Polaridade'].apply(getAnalysis)

df_coment = df_coment.drop('Legenda', axis=1)
df_coment = df_coment.drop('Likes', axis=1)

df_coment['Polaridade'] = df_coment['Polaridade'].astype(float)
df_coment['Subjetividade'] = df_coment['Subjetividade'].astype(float)
df_coment['Polaridade'] = round(df_coment['Polaridade'], 3)
df_coment['Subjetividade'] = round(df_coment['Subjetividade'], 3)
df_coment['Emojis'] = df_coment['Emojis'].apply(lambda x: ''.join(x))

df_coment.to_csv('Data/FATO_COMENTARIOS_EMOJIS.csv', index=False, encoding='utf-8')
df_coment

### 2.5.2. Limpeza e tratamento dos dados das legendas

In [85]:
df = pd.read_csv('Data/TABELA_POSTS_COMENTARIOS_MUNICIPIOS.csv', delimiter = ',')
df_legenda = df.copy()
df_legenda = df_legenda[df_legenda['Legenda'].notnull()]
df_legenda['Legenda'] = df_legenda['Legenda'].apply(lambda x: x.split())

for i in range(len(df_legenda['Legenda'])):
    j=0
    for j in range(len(df_legenda['Legenda'].values[i])):
        if (df_legenda['Legenda'].values[i][j].startswith('@')):
            df_legenda['Legenda'].values[i][j] = ''

df_legenda['Legenda'] = df_legenda['Legenda'].apply(lambda x: " ".join(x))
df_legenda['Legenda'] = df_legenda['Legenda'].apply(remove_emojis)
df_legenda['Legenda'] = df_legenda['Legenda'].apply(cleanTxt)
df_legenda = df_legenda[df_legenda['Legenda']!='']
df_legenda = df_legenda[df_legenda['Legenda']!=' ']
df_legenda = df_legenda[df_legenda['Legenda'].notnull()]

#print(f'Iteração {aux+1}/{len(df_dados_clientes.index)}')

df_legenda = df_legenda.drop('Comentarios', axis=1)

#df_legenda.to_csv('FATO_LEGENDAS.csv', index=False, encoding='utf-8')
df_legenda

,Municipio,UF,Post/Coment,Data,Usuario,Legenda,Likes
0,Ijui,RS,Post,2022-09-16 12:02:41,prefeituradeijui,"Ponto Facultativo! Segunda-feira, 19, órgãos p...",22.0
1,Ijui,RS,Post,2022-09-16 12:01:01,prefeituradeijui,Evento Regional Feira da Primavera acontece ne...,69.0
2,Ijui,RS,Post,2022-09-16 11:59:20,prefeituradeijui,Oportunidade de qualificação profissional! Cur...,4.0
3,Ijui,RS,Post,2022-09-16 11:58:21,prefeituradeijui,Drenagem no Osvaldo Aranha! Serviço de calçame...,8.0
4,Ijui,RS,Post,2022-09-16 11:56:54,prefeituradeijui,Tecnologia para a saúde! UBS e agentes comunit...,123.0
5,Ijui,RS,Post,2022-09-16 11:53:23,prefeituradeijui,Farmácia vai à escola! Estudantes recebem orie...,19.0
6,Ijui,RS,Post,2022-09-15 13:49:58,prefeituradeijui,Mutirão de limpeza! Prefeito Andrei Cossetin a...,112.0
8,Ijui,RS,Post,2022-09-15 13:12:43,prefeituradeijui,Bola na rede! Domingo tem rodada do Campeonato...,4.0
9,Ijui,RS,Post,2022-09-15 12:35:08,prefeituradeijui,Letramento digital! SMEd avança no uso de recu...,12.0
10,Ijui,RS,Post,2022-09-15 12:31:40,prefeituradeijui,Estradas recuperadas! Linhas 7 e 8 norte receb...,22.0
